## Import Libraries

In [57]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.flow.flow import Flow, start, listen  
from langchain_community.tools import DuckDuckGoSearchRun
import textwrap



##  Disable CrewAI's telemetry to prevent data collection and errors

In [58]:
os.environ["CREWAI_TELEMETRY"] = "false"
os.environ["CREWAI_TELEMETRY_DEBUG"] = "0"  

## Load the  LLM


In [59]:
from crewai import LLM


llm = LLM(
    model="groq/qwen/qwen3-32b",
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1",
    temperature=0.2,
    max_tokens=1000,
    num_retries=5,
    request_timeout=90
)

## Load  DuckduckGo Search tool

In [60]:
import time
import random
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Topic for the LLM

In [61]:
topic = "2024 Honda Civic review"

## Defining all Agents

In [62]:
# #  Agent 1: Reviewer A with an Edmunds bias
reviewer1 = Agent(
    role="Reviewer A (Edmunds bias)",
    goal="Search Edmunds for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Automotive editor who summarizes objectively.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False  

)
#  Agent 2: Reviewer B with a Car and Driver bias
reviewer2 = Agent(
    role="Reviewer B (Car and Driver bias)",
    goal="Search Car and Driver for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Performance-focused reviewer with balanced takes.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)
# Agent 3: Reviewer C with a Top Gear/other magazine bias
reviewer3 = Agent(
    role="Reviewer C (Top Gear / other mags bias)",
    goal="Search Top Gear or other reputable magazines and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Keeps things practical for buyers.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)

#  Agent 4: Final Synthesizer, to combine the findings
synthesizer = Agent(
    role="Final Synthesizer",
    goal="Merge the three reviewer notes into one clear, non-redundant verdict.Only output the final answer without explaining the steps",
    backstory="Senior editor who reconciles differing opinions.",
    tools=[],  # no web/tools; purely synthesis
    llm=llm,
    allow_delegation=False, 
    verbose=False
)


## Giving tasks descriptions 

In [63]:


# Tasks for the three retrievers
t1 = Task(
    description=f"Topic: {topic}\nSearch site:edmunds.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer1,
    expected_output="5 bullets + verdict + links"
)
t2 = Task(
    description=f"Topic: {topic}\nSearch site:caranddriver.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer2,
    expected_output="5 bullets + verdict + links"
)
t3 = Task(
    description=f"Topic: {topic}\nSearch (site:topgear.com OR site:autocar.co.uk OR site:whatcar.com) {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer3,
    expected_output="5 bullets + verdict + links"
)

# Running  ONLY the three retrievers in PARALLEL
t1.async_execution = True
t2.async_execution = True
t3.async_execution = True

# Final summarizer task (sequential) that COMBINES t1+t2+t3
t_final = Task(
    description=(
        "Combine the three reviewer outputs (given in context) into:\n"
        "- 4-6 consensus bullets (no duplicates)\n"
        "- One verdict paragraph (3-5 sentences) for buyers\n"
        "- 'Where reviewers agree' and 'Where they differ' (1-2 bullets each)\n"
        "Keep it neutral, specific, and text-only."
    ),
    agent=synthesizer,           #  4th agent
    expected_output="Bullets + verdict paragraph + agreement/differences",
    context=[t1, t2, t3]         # this pulls outputs from the three retrievers
)
from IPython.display import display, Markdown

# One crew: 3 parallel retrievers  final sequential summarizer
crew = Crew(
    agents=[reviewer1, reviewer2, reviewer3, synthesizer],
    tasks=[t1, t2, t3, t_final],   # final LAST
    process=Process.sequential,
    verbose=False,
)

result = crew.kickoff()

# Show all outputs 
outs = getattr(result, "tasks_output", []) or []
for i, t in enumerate(outs):
    role = getattr(getattr(t, "agent", None), "role", f"Task {i}")
    body = getattr(t, "raw", None) or getattr(t, "output", None) or ""
    display(Markdown(f"### {i}: {role}\n\n{body}"))

### 0: Task 0

- **Fuel Efficiency**: Up to 32 mpg on the highway, making it one of the most efficient compact cars.  
- **Tech & Features**: Standard Apple CarPlay, Android Auto, and a 7-inch touchscreen with optional upgrades to 12.3-inch displays.  
- **Driving Experience**: Peppy 2.0L engine (160 hp) with smooth CVT transmission and responsive handling.  
- **Interior Comfort**: Roomy cabin with supportive seats and decent cargo space for its class.  
- **Reliability**: Strong build quality and a 10-year/100,000-mile powertrain warranty.  

**Verdict**: The 2024 Honda Civic remains a top compact car choice, balancing efficiency, tech, and practicality.  

**Links**:  
https://www.edmunds.com/2024-honda-civic-review-rating/  
https://www.edmunds.com/honda-civic/

### 1: Task 1

**2024 Honda Civic Review (Car and Driver):**  
- **Pros:** Improved 1.5L turbo engine delivers 182 hp, refined 10-speed automatic transmission, enhanced interior tech with 12.3-inch digital gauge cluster, sharper handling with updated suspension, and competitive starting price.  
- **Cons:** Rear seat legroom remains tight, base model lacks advanced driver-assist features, plastic-heavy interior feels less premium, exhaust note is less engaging compared to competitors, and optional tech packages add significant cost.  
- **Verdict:** The 2024 Civic offers strong performance and value but falls short in cabin comfort and tech integration. A solid choice for drivers prioritizing efficiency and handling.  
- **Links:**  
  https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-first-drive-review  
  https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-ultimate-review

### 2: Task 2

- **Responsive 1.5T engine** with smooth 10-speed auto transmission, delivering brisk acceleration and efficiency.  
- **Sharp handling** and well-tuned suspension make it a joy to drive, with precise steering and minimal body roll.  
- **Modern tech suite** includes a 10.2-inch touchscreen, Apple CarPlay/Android Auto, and driver-assist features.  
- **Roomy and practical interior** with quality materials, though rear legroom is slightly compromised by sporty seating.  
- **Impressive fuel economy** (up to 40 mpg combined) and low emissions for a sporty compact.  

**Verdict**: The 2024 Honda Civic strikes a near-perfect balance between driving fun, tech, and practicality—still the benchmark for compact sedans.  

https://www.topgear.com/car-reviews/honda/civic-2024-review  
https://www.autocar.co.uk/car-reviews/honda/civic-2024

### 3: Task 3

<think>
Okay, let's tackle this. The user wants me to combine three reviewer notes into a structured answer with consensus bullets, a verdict paragraph, and sections on agreement and differences. First, I need to parse each review's key points.

Starting with the context provided: Fuel efficiency up to 32 mpg, tech features like Apple CarPlay, a 2.0L engine, interior comfort, and reliability with a 10-year warranty. The initial verdict says it's a top compact car.

Now, the Car and Driver review mentions a 1.5L turbo engine with 182 hp, 10-speed auto, improved interior tech, sharper handling, but points out tight rear legroom and lack of driver-assist in base models. Their verdict is positive on performance and value but notes shortcomings in comfort and tech.

The third review (from Top Gear?) highlights the 1.5T engine, sharp handling, modern tech, roomy interior with some compromises on rear legroom, and 40 mpg combined. Their verdict is that it balances driving fun, tech, and practicality.

Looking for consensus bullets: All three mention improved engine performance (either 1.5L turbo or 2.0L), tech features like Apple CarPlay, handling improvements, fuel efficiency (32-40 mpg), and some mention of interior comfort. The 10-speed transmission is mentioned in Car and Driver and the third review. The 10-year warranty is in the context but not in the other reviews, so maybe not consensus. Need to check if that's part of the context or not. The context includes a 10-year/100k powertrain warranty, but the other reviews don't mention it. Since the user wants consensus bullets, maybe only include those mentioned in all three reviews. Wait, the context is part of the input, so maybe the three reviews are the ones from Edmunds, Car and Driver, and Top Gear. The context's info is part of the given data. Hmm, the user says "three reviewer outputs" but the context includes the initial criteria and then three separate reviews. Wait, the user provided three different reviews: the first is the context (Edmunds), then Car and Driver, then Top Gear. So the three reviews are Edmunds, Car and Driver, and Top Gear. So need to check which points are common across all three.

Looking for consensus bullets:

1. Engine: All three mention improved engine performance. Edmunds says 2.0L 160hp, Car and Driver 1.5L turbo 182hp, Top Gear 1.5T. So maybe "Improved engine options (1.5L turbo and 2.0L) with enhanced power and efficiency."

2. Transmission: Car and Driver and Top Gear mention 10-speed auto, Edmunds mentions CVT. So maybe not consensus on transmission type. Hmm. Maybe not include transmission in consensus.

3. Tech: All three mention Apple CarPlay and Android Auto. Car and Driver mentions 12.3-inch digital cluster as optional, Top Gear mentions 10.2-inch touchscreen. Edmunds mentions 7-inch standard with optional 12.3. So consensus on tech features like smartphone integration.

4. Handling: All three mention improved handling. Car and Driver says sharper with updated suspension, Top Gear says sharp handling and well-tuned suspension, Edmunds says responsive handling. So consensus on handling.

5. Fuel economy: All three mention high mpg (32-40). So consensus on fuel efficiency.

6. Interior comfort: Edmunds says roomy, Car and Driver mentions tight rear legroom, Top Gear says roomy but compromised rear legroom. So maybe not consensus. But maybe "Roomy and practical interior with some compromises in rear legroom."

Wait, need to check if all three mention interior comfort. Edmunds says roomy cabin, Car and Driver says tight rear legroom, Top Gear says roomy but compromised. So maybe consensus on interior being practical but with rear legroom issues. So that could be a bullet.

So possible consensus bullets:

- Improved engine options (1.5L turbo and 2.0L) with enhanced power and efficiency.
- Standard Apple CarPlay and Android Auto with optional larger touchscreen/digital gauge clusters.
- Sharp handling and well-tuned suspension for responsive driving dynamics.
- Impressive fuel economy (up to 40 mpg combined) with strong emissions performance.
- Roomy and practical interior with supportive seating, though rear legroom is limited in some trims.

Verdict paragraph needs to be a synthesis of all three verdicts. The initial context's verdict says it's a top choice balancing efficiency, tech, practicality. Car and Driver says strong performance and value but lacks in comfort and tech. Top Gear says